Here is the corrected markdown with standard LaTeX formatting.

**Purpose.**
1) derives and visualizes the **feasibility ratio** $\rho_N$,
2) quantifies **combinatorial ambiguity** via linear-extension counts/estimates,
3) demonstrates **dynamic breakdown** of L-flow for larger $N$ with Monte Carlo,
4) exports stable figures/JSON for manuscript and downstream notebooks.


## 0. Executive Summary
- **Claim:** $N=4$ is the maximal complexity at which the L-filter reliably yields a stable actuality.
- **Evidence:** (i) **Analytic scarcity**: $\rho_N = \frac{N!}{2^{N(N-1)/2}}$ collapses with $N$.
- (ii) **Combinatorial ambiguity**: the number of linear extensions of typical partial orders grows rapidly, complicating completion.
- (iii) **Dynamic breakdown**: simulated L-flow quickly stalls (rarely yields a unique total) for $N \ge 5$.

## 1. Analytic Scarcity: Feasibility Ratio \(\rho_N\)
Define
$$\rho_N = \frac{N!}{2^{\,N(N-1)/2}},$$
the fraction of all pairwise sign choices that correspond to a consistent total order. We plot \(\rho_N\) and overlay a Stirling-based asymptotic.

In [1]:
import matplotlib.pyplot as plt
from math import factorial, pi, e, sqrt

def rho_exact(N):
    return factorial(N) / (2**(N*(N-1)//2))

def rho_stirling(N):
    # Stirling: N! ~ sqrt(2πN)(N/e)^N
    return (sqrt(2*pi*N)*(N/e)**N) / (2**(N*(N-1)/2))

Ns = list(range(2,13))
exact = [rho_exact(n) for n in Ns]
approx = [rho_stirling(n) for n in Ns]

plt.figure(figsize=(6,4))
plt.plot(Ns, exact, marker='o', label='exact')
plt.plot(Ns, approx, marker='x', label='Stirling approx')
plt.yscale('log')
plt.xlabel('N')
plt.ylabel('rho_N (log scale)')
plt.title('Feasibility ratio rho_N vs N')
plt.legend()
plt.tight_layout()
plt.savefig('/mnt/data/00d_rhoN_exact_vs_stirling.png', dpi=160)
plt.show()
list(zip(Ns, exact, approx))[:5]

## 2. Combinatorial Ambiguity: Linear Extensions
For a DAG (partial order) on \(N\) elements, the number of **linear extensions** counts distinct total completions. Exact counting is \#P-complete in general, so we do:
- exact counts for small \(N\) by enumeration,
- a **Monte Carlo estimator** (Karzanov–Khachiyan–style sampling) for larger \(N\) to gauge typical growth.

We use random DAGs generated by orienting edges of an Erdos–Renyi graph and deleting cycles.

In [2]:
from collections import deque
import numpy as np

def random_dag(N, p=0.25, seed=None):
    rng = random.Random(seed)
    # random ordering to induce acyclicity, then add forward edges with prob p
    order = list(range(N))
    rng.shuffle(order)
    pos = {order[i]: i for i in range(N)}
    E = []
    for i in range(N):
        for j in range(N):
            if i==j: continue
            if pos[i] < pos[j] and rng.random() < p:
                E.append((i,j))
    return E

def count_linear_extensions_exact(N, E, limit=None):
    # exact backtracking (feasible up to ~N=8)
    adj=[[] for _ in range(N)]
    indeg=[0]*N
    for u,v in E:
        adj[u].append(v)
        indeg[v]+=1
    avail=[i for i in range(N) if indeg[i]==0]
    count=0
    stack=[(tuple(), tuple(sorted(avail)), tuple(indeg))]
    while stack:
        pref, avail, indeg_t = stack.pop()
        if len(pref)==N:
            count+=1
            if limit and count>=limit:
                return count
            continue
        for x in avail:
            new_indeg=list(indeg_t)
            new_avail=list(avail)
            new_avail.remove(x)
            for w in adj[x]:
                new_indeg[w]-=1
                if new_indeg[w]==0:
                    new_avail.append(w)
            stack.append((pref+(x,), tuple(sorted(new_avail)), tuple(new_indeg)))
    return count

def estimate_linear_extensions_mc(N, E, trials=2000, seed=0):
    # Knuth-style importance sampling: multiply branching factors along random linear extension paths
    rng = random.Random(seed)
    adj=[[] for _ in range(N)]
    indeg=[0]*N
    for u,v in E:
        adj[u].append(v)
        indeg[v]+=1
    ests=[]
    for _ in range(trials):
        indeg_t = indeg[:]
        avail=[i for i in range(N) if indeg_t[i]==0]
        prod=1.0
        for _k in range(N):
            b=len(avail)
            if b==0: 
                prod=0.0; break
            prod*=b
            x = rng.choice(avail)
            avail.remove(x)
            for w in adj[x]:
                indeg_t[w]-=1
                if indeg_t[w]==0:
                    avail.append(w)
        ests.append(prod)
    return float(np.mean(ests)), float(np.std(ests))

# sanity on small N
E5 = random_dag(5, p=0.3, seed=1)
exact5 = count_linear_extensions_exact(5, E5)
mc5, sd5 = estimate_linear_extensions_mc(5, E5, trials=2000)
exact5, mc5, sd5

### Typical growth with N
We sample random DAGs and record the (log) linear-extension estimates to show growth and dispersion.

In [3]:
def summary_linear_ext(Ns=(4,5,6,7,8,9), p=0.25, samples=60, seed=0):
    rng = random.Random(seed)
    rows=[]
    for N in Ns:
        for s in range(samples):
            E = random_dag(N, p=p, seed=rng.randint(0,10**9))
            m, sd = estimate_linear_extensions_mc(N, E, trials=600, seed=rng.randint(0,10**9))
            rows.append((N, m, sd))
    return rows

rows = summary_linear_ext(Ns=(4,5,6,7,8), p=0.25, samples=50, seed=2)
import pandas as pd
df = pd.DataFrame(rows, columns=['N','mc_mean','mc_sd'])
df['log10_mean'] = np.log10(np.maximum(1, df['mc_mean']))
df.head(), df.groupby('N')['log10_mean'].describe()

In [4]:
plt.figure(figsize=(6,4))
for N,g in df.groupby('N'):
    jitter = (np.random.rand(len(g))-0.5)*0.1
    plt.scatter([N+jj for jj in jitter], g['log10_mean'], alpha=0.6, label=f'N={N}')
plt.xlabel('N')
plt.ylabel('log10(est. # linear extensions)')
plt.title('Combinatorial ambiguity grows with N')
plt.tight_layout()
plt.savefig('/mnt/data/00d_linear_extensions_growth.png', dpi=160)
plt.show()

## 3. Dynamic Breakdown: L-flow Efficiency with N
We approximate an L-flow: from a random partial order (DAG), repeatedly add **resolving edges** consistent with acyclicity until no ambiguity remains or we stall (non-unique choices persist). We measure the fraction of runs that terminate in a **unique total order**.

In [5]:
def lflow_unique_completion_rate(N, trials=200, p=0.25, seed=0):
    rng = random.Random(seed)
    success=0
    for _ in range(trials):
        E = random_dag(N, p=p, seed=rng.randint(0,10**9))
        # while multiple linear extensions, add a random consistent edge to reduce ambiguity
        # heuristic: sample pairs (i,j) with i before j in many sampled extensions and add (i,j)
        # stop early if too many steps
        steps=0
        # quick function to sample linear extensions and get pairwise precedence tallies
        def sample_ext_tally(samples=150):
            adj=[[] for _ in range(N)]
            indeg=[0]*N
            for u,v in E:
                adj[u].append(v)
                indeg[v]+=1
            tally=np.zeros((N,N), dtype=int)
            for s in range(samples):
                indeg_t=indeg[:]
                avail=[i for i in range(N) if indeg_t[i]==0]
                ext=[]
                while avail:
                    x = rng.choice(avail)
                    avail.remove(x)
                    ext.append(x)
                    for w in adj[x]:
                        indeg_t[w]-=1
                        if indeg_t[w]==0:
                            avail.append(w)
                # tally precedence
                pos={ext[i]:i for i in range(N)}
                for i in range(N):
                    for j in range(N):
                        if i==j: continue
                        if pos[i] < pos[j]:
                            tally[i,j]+=1
            return tally
        # loop: try to add edges toward consensus
        while steps<40:
            # check if unique total
            cnt = count_linear_extensions_exact(N,E,limit=2)
            if cnt==1:
                success+=1; break
            tally = sample_ext_tally(120)
            # find undecided pair with strongest majority and add it if acyclic
            best=None; best_margin=0
            for i in range(N):
                for j in range(N):
                    if i==j: continue
                    if (i,j) in E or (j,i) in E: continue
                    m = tally[i,j] - tally[j,i]
                    if m>best_margin:
                        best_margin=m; best=(i,j)
            if best is None or best_margin==0: # stalled
                break
            # add edge if it keeps acyclic
            E2=E+[(best[0],best[1])]
            # acyclicity check
            indeg=[0]*N; adj=[[] for _ in range(N)]
            for u,v in E2:
                adj[u].append(v); indeg[v]+=1
            q=deque([i for i in range(N) if indeg[i]==0]); seen=0
            while q:
                u=q.popleft(); seen+=1
                for w in adj[u]:
                    indeg[w]-=1
                    if indeg[w]==0:
                        q.append(w)
            if seen==N:
                E=E2; steps+=1
            else:
                break
    return success/trials

rates=[]
for N in [3,4,5,6]:
    r = lflow_unique_completion_rate(N, trials=60, p=0.25, seed=1)
    rates.append((N,r))
rates

In [6]:
Ns=[r[0] for r in rates]; R=[r[1] for r in rates]
plt.figure(figsize=(6,4))
plt.plot(Ns, R, marker='o')
plt.ylim(0,1)
plt.xlabel('N')
plt.ylabel('Pr(unique total order | L-flow heuristic)')
plt.title('Dynamic breakdown with N: success probability')
plt.tight_layout()
plt.savefig('/mnt/data/00d_lflow_success_prob.png', dpi=160)
plt.show()